In [2]:
import pandas as pd
import numpy as np


In [3]:
MONTHS_GEN = {
    1: "января",
    2: "февраля",
    3: "марта",
    4: "апреля",
    5: "мая",
    6: "июня",
    7: "июля",
    8: "августа",
    9: "сентября",
    10: "октября",
    11: "ноября",
    12: "декабря"
}

In [11]:
data = pd.read_csv('weather_data.csv', sep=',').drop(['text'], axis='columns')

In [31]:
data.head()

(11875, 120)

In [13]:
month_list = list(MONTHS_GEN.values())
month_list.append('month')


In [14]:
data = data.drop(month_list, axis='columns')
data_start = data.drop(['day_end'], axis='columns')

In [15]:
data_start.head()

,21-22,8-10,8-9,23-27,16-20,26-29,08-09,9,20-25,10-12,...,27,11-13,14-16,15-17,3-4,26-28,22-24,25,12-14,day_start
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,9
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,9
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,9


In [33]:
data_start = data_start.drop(data_start[data_start['day_start'] > 31].index)
data_start = data_start.drop(data_start[data_start['day_start'] < 1].index)

In [42]:

start_x= data_start.drop('day_start', axis=1)
start_y = data_start['day_start'].astype('int')


In [43]:
start_x.shape, start_y.shape

((11727, 118), (11727,))

In [44]:
start_y.head()

0     9
1    10
2     9
3     8
4     9
Name: day_start, dtype: int32

In [45]:
start_y.describe()

count    11727.000000
mean        15.392428
std          8.830100
min          1.000000
25%          8.000000
50%         15.000000
75%         23.000000
max         31.000000
Name: day_start, dtype: float64

In [49]:
from sklearn.tree import DecisionTreeClassifier
first_tree = DecisionTreeClassifier(random_state=17)

In [51]:
from sklearn.model_selection import train_test_split
start_x_train, start_x_valid, start_y_train, start_y_valid = train_test_split(start_x, start_y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [52]:
start_x_train.shape, start_x_valid.shape

((8208, 118), (3519, 118))

In [56]:
from sklearn.model_selection import  cross_val_score
cross_val_score(first_tree, start_x_train, start_y_train, cv=5)

array([0.8727162 , 0.86723508, 0.86419001, 0.87507617, 0.87446679])

In [57]:
from sklearn.neighbors import KNeighborsClassifier

In [60]:
first_knn = KNeighborsClassifier()
np.mean(cross_val_score(first_knn, start_x_train, start_y_train, cv=5))

0.8717110493067045

In [133]:
%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators=25, random_state=17,
    
    )
rf.fit(start_x_train, start_y_train)

Wall time: 0 ns


RandomForestClassifier(max_features='log2', n_estimators=25, random_state=17)

In [134]:
%time
rf_valid_pred = rf.predict(start_x_valid)


Wall time: 0 ns


In [135]:
from sklearn.metrics import accuracy_score
accuracy_score(start_y_valid, rf_valid_pred)

0.8815004262574595

In [128]:
from sklearn.metrics import confusion_matrix
confusion_matrix(start_y_valid, rf_valid_pred)


array([[ 99,   2,   1,   1,   3,   0,   0,   1,   2,   0,   1,   2,   1,
          2,   3,   1,   2,   1,   2,   3,   1,   3,   1,   2,   4,   2,
          1,   3,   1,   1,   1],
       [  1, 107,   2,   4,   0,   1,   1,   1,   0,   6,   0,   4,   1,
          2,   1,   0,   1,   0,   1,   0,   1,   2,   1,   0,   1,   0,
          0,   1,   0,   1,   0],
       [  0,   1,  88,   2,   2,   0,   0,   1,   0,   5,   1,   0,   0,
          0,   0,   0,   2,   0,   0,   2,   2,   1,   1,   0,   1,   2,
          1,   0,   2,   0,   0],
       [  0,   0,   2,  86,   0,   0,   0,   0,   1,   2,   0,   0,   1,
          1,   1,   1,   0,   0,   1,   2,   0,   1,   0,   0,   0,   0,
          0,   1,   1,   1,   0],
       [  1,   1,   1,   2,  99,   0,   0,   0,   0,   2,   1,   3,   0,
          0,   0,   0,   0,   0,   0,   0,   1,   1,   1,   1,   0,   1,
          1,   0,   0,   0,   0],
       [  0,   0,   0,   0,   1, 112,   0,   1,   0,   3,   2,   0,   0,
          1,   0,   0,   0,